In [ ]:
import ee
import pandas as pd
import time

ee.Initialize(project='environment-change-detection')
df = pd.read_csv("zone_features_with_forest.csv")

def get_temperature(lat, lon, start="2023-01-01", end="2023-12-31"):
    point = ee.Geometry.Point([lon, lat])
    try:
        image = ee.ImageCollection("MODIS/006/MOD11A2") \
                    .filterDate(start, end) \
                    .select("LST_Day_1km") \
                    .mean() \
                    .multiply(0.02).subtract(273.15)
        result = image.reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()
        return round(result, 2) if result is not None else None
    except:
        return None

def get_NO2(lat, lon, start="2023-01-01", end="2023-12-31"):
    point = ee.Geometry.Point([lon, lat])
    try:
        image = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
                    .filterDate(start, end) \
                    .select("NO2_column_number_density") \
                    .mean()
        result = image.reduceRegion(ee.Reducer.mean(), point, 1000).get("NO2_column_number_density").getInfo()
        return round(result, 6) if result is not None else None
    except:
        return None

temp_mine_list = []
temp_forest_list = []
no2_mine_list = []
no2_forest_list = []

for i, row in df.iterrows():
    print(f"Processing zone {i+1}/{len(df)}: {row['Zone']}")
    lat_mine, lon_mine = row['Lat'], row['Long']
    lat_forest, lon_forest = row['ForestLat'], row['ForestLon']
    
    temp_mine_list.append(get_temperature(lat_mine, lon_mine))
    temp_forest_list.append(get_temperature(lat_forest, lon_forest))
    no2_mine_list.append(get_NO2(lat_mine, lon_mine))
    no2_forest_list.append(get_NO2(lat_forest, lon_forest))

    time.sleep(0.5)

df['Temp_Mine'] = temp_mine_list
df['Temp_Forest'] = temp_forest_list
df['NO2_Mine'] = no2_mine_list
df['NO2_Forest'] = no2_forest_list

df.to_csv("zone_features_enriched.csv", index=False)
print("✅ zone_features_enriched.csv saved with temp and NO₂")